In [2]:
!pip install openai
!pip install langchain
!pip install unstructured
!pip install -U langchain-openai
!pip install langchain_community
!pip install PyMuPDF
!pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.1 MB/s eta 0:00:00
  Preparing metada

In [3]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [18]:
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone
!pip install langchain langchain_openai

In [5]:
llm = OpenAI(temperature=0.9, max_tokens=500)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [8]:
from langchain.docstore.document import Document
import fitz  # PyMuPDF
import os

def extract_text_from_pdf(pdf_Path):
    document = fitz.open(pdf_Path)
    data = ""
    for page in document:
        data += page.get_text()
    return data

pdf_Path = '/content/asd.pdf'
data = extract_text_from_pdf(pdf_Path)
data =  Document(page_content=data, metadata={"source": "local"})

In [9]:
# CharacterTextSplitter split based on characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents([data])

In [10]:
# Create the embeddings of the chunks using openAIEmbeddings
import openai
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
from langchain_pinecone import Pinecone
import os
os.environ['PINECONE_API_KEY'] = "aa0b76fb-3f76-4a99-8e2b-02ed5611b97a"
indexName = "msindex"

docsearch = Pinecone.from_documents(index_name = indexName,documents = docs,embedding = embeddings)

In [19]:
from IPython.display import display

import ipywidgets as widgets
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [47]:
memory = ConversationBufferWindowMemory(memory_key='chat_history',output_key='answer')
query = "In which region they will focus ?"
docs = docsearch.similarity_search(query)
# print(docs[0].page_content)



chain = RetrievalQAWithSourcesChain.from_llm(llm=llm,memory=memory, retriever=docsearch.as_retriever())

# chain = RetrievalQAWithSourcesChain.from_llm(
#     llm=llm,
#     memory=memory,
#     retriever=docsearch.as_retriever(),
#     output_key='answer'
# )
result=chain({"question": query} ,return_only_outputs=False)



display(widgets.HTML(f'<b><font color="blue">Output :</font></b> {result["answer"]}'))




HTML(value='<b><font color="blue">Output :</font></b>  The government will focus on the eastern region.\n')

In [48]:
chain.memory.load_memory_variables({})

{'chat_history': 'Human: In which region they will focus ?\nAI:  The government will focus on the eastern region.\n'}